In [1]:
import pandas as pd

## Input: A single excel file which concatenated all blocks (112) halfhourly data, each row representing invidual block & LCLid halfhourly data (1 row represents one unique LCLid in one block)

## Output: A single excel file, energy consumption aggregated by date, LCLid aggregated as unique count per hour, block id aggregated as unique block per hour.

In [2]:
destination_path = 'C:/Users/chiny/cjwen/ITI110/'


In [3]:
df = pd.read_csv(destination_path + "Raw_data/concatenated_halfhourly_dataset.csv")

In [4]:
df

,LCLid,tstp,energy(kWh/hh),block_id
0,MAC000002,2012-10-12 00:30:00.0000000,0,block_0
1,MAC000002,2012-10-12 01:00:00.0000000,0,block_0
2,MAC000002,2012-10-12 01:30:00.0000000,0,block_0
3,MAC000002,2012-10-12 02:00:00.0000000,0,block_0
4,MAC000002,2012-10-12 02:30:00.0000000,0,block_0
...,...,...,...,...
167817016,MAC005019,2014-02-27 22:00:00.0000000,0.129,block_99
167817017,MAC005019,2014-02-27 22:30:00.0000000,0.095,block_99
167817018,MAC005019,2014-02-27 23:00:00.0000000,0.061,block_99
167817019,MAC005019,2014-02-27 23:30:00.0000000,0.054,block_99


In [5]:
# Remove outlier meters
to_remove = ['MAC000004', 
 'MAC000037', 
 'MAC000197', 
 'MAC000325', 
 'MAC000399', 
 'MAC000895', 
 'MAC001309', 
 'MAC001976', 
 'MAC002050', 
 'MAC002110', 
 'MAC002136', 
 'MAC002564', 
 'MAC002594', 
 'MAC002858', 
 'MAC002873', 
 'MAC002976', 
 'MAC003156', 
 'MAC003327', 
 'MAC003627', 
 'MAC004067', 
 'MAC004672', 
 'MAC004731', 
 'MAC004735', 
 'MAC005033', 
 'MAC005069', 
 'MAC005313']

In [6]:
df = df[~df["LCLid"].isin(to_remove)] #remove outlier meters

In [9]:
# df[df["LCLid"]=="MAC000004"]

In [8]:
#Format data
df['energy(kWh/hh)'] = pd.to_numeric(df['energy(kWh/hh)'], errors='coerce')
df['tstp'] = pd.to_datetime(df['tstp'])

C:\Users\chiny\AppData\Local\Temp\ipykernel_144960\1440308734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['energy(kWh/hh)'] = pd.to_numeric(df['energy(kWh/hh)'], errors='coerce')
C:\Users\chiny\AppData\Local\Temp\ipykernel_144960\1440308734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tstp'] = pd.to_datetime(df['tstp'])


In [9]:
#extract time features
df["date"] = df["tstp"].dt.date
df["month"]= df["tstp"].dt.month
df["hour"]= df["tstp"].dt.hour

C:\Users\chiny\AppData\Local\Temp\ipykernel_144960\4228368789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = df["tstp"].dt.date
C:\Users\chiny\AppData\Local\Temp\ipykernel_144960\4228368789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["month"]= df["tstp"].dt.month


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167068985 entries, 0 to 167817020
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   LCLid           object        
 1   tstp            datetime64[ns]
 2   energy(kWh/hh)  float64       
 3   block_id        object        
 4   date            object        
 5   month           int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 8.1+ GB


In [11]:
agg_df = df.groupby(['date']).agg({
            'date': 'max',
            'month':'max',
            # 'hour': 'max',
            'LCLid': lambda x: x.nunique(),
            'block_id': lambda y: y.nunique(),
            'energy(kWh/hh)': 'sum'
        }).reset_index(drop=True)

In [14]:
agg_df.rename(columns={"LCLid": "count_LCLid", "block_id": "count_block_id","energy(kWh/hh)":"sum_energy_by_day"},inplace=True)

In [23]:
agg_df.tail(2)

,date,month,count_LCLid,count_block_id,sum_energy_by_day
827,2014-02-27,2,4969,112,51647.284998
828,2014-02-28,2,4966,112,1042.087000


In [20]:
pd.to_datetime(agg_df["date"], format = "%Y-%m-%d")

0     2011-11-23
1     2011-11-24
2     2011-11-25
3     2011-11-26
4     2011-11-27
         ...    
824   2014-02-24
825   2014-02-25
826   2014-02-26
827   2014-02-27
828   2014-02-28
Name: date, Length: 829, dtype: datetime64[ns]

In [21]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               829 non-null    object 
 1   month              829 non-null    int32  
 2   count_LCLid        829 non-null    int64  
 3   count_block_id     829 non-null    int64  
 4   sum_energy_by_day  829 non-null    float64
dtypes: float64(1), int32(1), int64(2), object(1)
memory usage: 29.3+ KB


In [22]:
agg_df.isnull().sum()

date                 0
month                0
count_LCLid          0
count_block_id       0
sum_energy_by_day    0
dtype: int64

In [24]:
agg_df.to_csv("1.3 aggregated_daily_dataset_without_block_id_02112024.csv",index=False)